In [167]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime
from selenium import webdriver
import time
from selenium.webdriver.firefox.firefox_binary import FirefoxBinary
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

In [146]:
def scrape_news(n):
    links = []
    #scroll = 5000
#     for k in range(4):
#         try:
#             driver = webdriver.Chrome('/Users/zhumingjia/chromedriver')
#             time.sleep(2)
#             driver.get(url2)
#             driver.execute_script("window.scrollBy(0, {})".format(str(scroll)))
#             scroll += 500
#             time.sleep(1)
#             driver.implicitly_wait(10)

#     driver = webdriver.Chrome('/Users/zhumingjia/chromedriver')
#     driver.get(url2)
#     driver.execute_script("window.scrollBy(0, {})".format(str(scroll)))
#     scroll += 200
#     scrolls = 10
#     while True:
#         scrolls -= 1
#         driver.execute_script("window.scrollTo(0, document.body.scrollHeight)")
#         time.sleep(3)
#         if scrolls < 0:
#             break

    for i in range(1, n):
        try:
            driver = webdriver.Chrome('/Users/zhumingjia/chromedriver') #use the path in your computer
            #driver.implicitly_wait(30)
            #get request
            time.sleep(2)
            driver.get(url2)
            time.sleep(2)
            for j in range(100):
                driver.execute_script("window.scrollBy(0, 500)")
                driver.implicitly_wait(20)

#             driver.execute_script("window.scrollBy(0, {})".format(str(scroll)))
            #scroll += 5
#             time.sleep(1)
#             driver.implicitly_wait(10)
        #keep clicking each news article and grab their url
            #driver.switch_to.frame(driver.find_element_by_class(""))
            elem = driver.find_element_by_xpath("//*[@id='summaryPressStream-0-Stream']/ul/li[{}]".format(i))
            #driver.implicitly_wait(30)
            #driver.implicitly_wait(15)
            #driver.execute_script("arguments[0].click();", elem)
        #driver.find_elements_by_class_name("Mb(5px)")
        #get the URL after click
        #     for e in elem:
        #         print(e.text)
            elem.click()
            #driver.implicitly_wait(30)
            links.append(driver.current_url)
            time.sleep(2)
        except:
            continue
#         except:
#             continue
            
    return links

In [146]:
def scrape_news_links(n):
    links = []

    #             driver.execute_script("window.scrollBy(0, {})".format(str(scroll)))
    #scroll += 5
    #             time.sleep(1)
    #             driver.implicitly_wait(10)
    #keep clicking each news article and grab their url
    #driver.switch_to.frame(driver.find_element_by_class(""))
    for i in range(1, n):
        try:
            driver = webdriver.Chrome('/Users/zhumingjia/chromedriver') #use the path in your computer
            #driver.implicitly_wait(30)
            #get request
            time.sleep(2)
            driver.get(url2)
            time.sleep(2)
            for j in range(600):
                driver.execute_script("window.scrollBy(0, 500)")
                driver.implicitly_wait(20)

            driver.implicitly_wait(40)
            #elem = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, "//*[@id='summaryPressStream-0-Stream']/ul/li[{}]/div/div/div[2]/h3/a".format(i))))
            elem = driver.find_element_by_xpath("//*[@id='summaryPressStream-0-Stream']/ul/li[{}]/div/div/div[2]/h3/a".format(i)).get_attribute("href")
            links.append(elem)
            print(elem)
            print(i)
            time.sleep(2)
        except:
            continue
    #driver.implicitly_wait(30)
    #driver.implicitly_wait(15)
    #driver.execute_script("arguments[0].click();", elem)
    #driver.find_elements_by_class_name("Mb(5px)")
    #get the URL after click
    #     for e in elem:
    #         print(e.text)
    # elem.click()
    # #driver.implicitly_wait(30)
    # links.append(driver.current_url)
    # time.sleep(2)
    return links

In [147]:
all_links = scrape_news(200)

In [157]:
def get_info(url):
    #grab the information
    response = requests.get(url, headers={'User-Agent': 'Custom'})
    soup = BeautifulSoup(response.text)
    news = soup.find('div', attrs={'class': 'caas-body'}).text
    headline = soup.find('h1').text 
    date = datetime.strptime(soup.find('time').text, "%B %d, %Y, %I:%M %p")
    date = date.strftime('%Y-%m-%d') 
    #combine all info into a list of columns
    columns = [news, headline,date]
    #give columns names
    column_names = ['News','Headline','Date']
    return dict(zip(column_names, columns))

In [162]:
df_dates = pd.DataFrame(columns=['name', 'date'])
for i in all_links:
    if i != 'https://finance.yahoo.com/quote/AAPL/press-releases?p=AAPL':
        d = get_info(i)
        new_file_name = 'apple_news_text_2/{}.txt'.format(d['Headline'])
        with open(new_file_name, 'w') as f:
            f.write(d['News'])
        dates = {'name':d['Headline'], 'date':d['Date']}
        df_dates = df_dates.append(dates, ignore_index=True)

In [163]:
df_dates

,name,date
0,Apple lands historic first Best Picture Oscar ...,2022-02-08
1,Apple Reports First Quarter Results,2022-01-27
2,Apple Announces Third Annual Apple Music Award...,2021-11-30
3,Apple Sues NSO Group to Curb the Abuse of Stat...,2021-11-23
4,Apple Introduces Apple Business Essentials,2021-11-10
5,Alex Gorsky Joins Apple’s Board of Directors,2021-11-09
6,Final Cut Pro and Logic Pro updated with power...,2021-10-18
7,Introducing M1 Pro and M1 Max: the most powerf...,2021-10-18
8,Game-changing MacBook Pro with M1 Pro and M1 M...,2021-10-18
9,CIBC launches five more Canadian Depositary Re...,2021-10-05
